In [1]:
from nltk import bigrams, word_tokenize
from nltk.util import ngrams
t='I am a boy'
tk=word_tokenize(t)
bg=bigrams(tk)#직전의 단어
ng=ngrams(tk,3)#특정 영역
list(bg),list(ng)

([('I', 'am'), ('am', 'a'), ('a', 'boy')],
 [('I', 'am', 'a'), ('am', 'a', 'boy')])

SS<문장>SE

In [3]:
#모델 바이그램 언어모델 : 앞에단어로 뒤에단어 예측
data2=ngrams(tk,2,pad_left=True,pad_right=True,left_pad_symbol="SS",right_pad_symbol="SE")
list(data2)

[('SS', 'I'), ('I', 'am'), ('am', 'a'), ('a', 'boy'), ('boy', 'SE')]

In [4]:
for i in data2:
    print(i[0],i[1])

In [5]:
#조건부확률
from nltk import ConditionalFreqDist
t='I am a boy'
t2='you are a boy'
tk=word_tokenize(t)
ng2=ngrams(tk,2,pad_left=True,pad_right=True,left_pad_symbol="SS",right_pad_symbol="SE")
fd=ConditionalFreqDist([(i[0],i[1])for i in ng2])

In [6]:
fd.conditions()#정답 접근 정보

['SS', 'I', 'am', 'a', 'boy']

In [7]:
fd['SS']

FreqDist({'I': 1})

In [8]:
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\류성빈\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\류성빈\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
from nltk.corpus import movie_reviews
data_l=[]
n=0
for i in movie_reviews.sents():#전체적인 구조를 볼수있음/여러문장이 이미 토큰화가 되어있음
    bg2=ngrams(i,2,pad_left=True,pad_right=True,left_pad_symbol="SS",right_pad_symbol="SE")    
    data_l+=[t for t in bg2]
    n+=1
#len(data_l)
#print(n)#조건의 개수
cfd= ConditionalFreqDist(data_l)

In [10]:
cfd["SS"].most_common(10)#너무많아 한정적으로 범위 줄여줌

[('the', 8071),
 ('.', 3173),
 ('it', 3136),
 ('i', 2471),
 ('but', 1814),
 ('and', 1735),
 ('he', 1672),
 ('in', 1659),
 ('this', 1651),
 ('there', 1298)]

In [11]:
cfd["the"].most_common(10)

[('film', 4542),
 ('movie', 2147),
 ('story', 985),
 ('most', 945),
 ('first', 902),
 ('same', 774),
 ('only', 665),
 ('end', 664),
 ('best', 642),
 ('audience', 620)]

In [12]:
cfd["movie"].most_common(10)

[('.', 802),
 (',', 659),
 ('is', 469),
 ("'", 234),
 ('that', 210),
 ('was', 123),
 ('and', 105),
 ('with', 87),
 ('to', 72),
 ('has', 69)]

In [13]:
cfd["."].most_common(10)

[('SE', 63404), ('"', 1854), (')', 535), ("'", 70), (']', 10), ("''", 3)]

In [16]:
from nltk.probability import ConditionalProbDist ,MLEProbDist
cpd = ConditionalProbDist(cfd,MLEProbDist)

In [18]:
cpd['the'].prob('movie')

0.0280547243528597

In [19]:
cpd['movie'].prob('.')

0.13897071564720154

In [20]:
cpd['.'].prob('the')

0.0

In [22]:
P(SS<문장>SE) = P(시작단어|SS)*P(|)*P(|)*P(|)*P(|)*P(SE|종료단어)
import numpy as np
def s_sc_f(x):#바이그램 언어 모형의 확률 연산 함수
    p = 0.0
    for i in range(len(x)-1):
        c=x[i]
        w=x[i+1]
        p+= np.log(cpd[c].prob(w)+np.finfo(float).eps)
    return np.exp(p)

In [23]:
test_data=['the','movie','.'] #실현 가능 문장
s_sc_f(test_data)

0.003898785120601922

In [24]:
test_data2=['movie','.','the'] #실현불가능 문장
s_sc_f(test_data2) #

3.085769765203191e-17

In [29]:
import random
random.seed(10)
cpd['SS'].generate()

'she'

In [30]:
cpd['she'].generate()

'and'

In [31]:
cpd['and'].generate()

'fine'

In [32]:
cpd['fine'].generate()

'effect'

In [33]:
cpd['effect'].generate()

';'

data="data+=문자열".join([])

In [ ]:
문자의 생성(조건부 확률을 기반한 )

In [ ]:
data=''
data+=문자열#더느리다 확인해야해서
str.join()#더 빠르다

In [39]:
st='SS'
all_str=[]
while True:
    import random
    random.seed(10)
    st=cpd[st].generate()
    all_str.append(st+' ')
    if st == 'SE':
        all_str.pop()
        break
''.join(all_str)

"she wasn ' s first part of these guys catch a hard 2 is . "

In [40]:
from nltk import ConditionalFreqDist#데이터가 얼만큼의 빈도수를 가지는지: 문맥별 단어의 빈도수측정
from nltk.util import ngrams #앞단어와 뒷단어를 갖고있는 data생성 토큰작업을하기위해
from nltk.probability import ConditionalProbDist#조건부 확률 추정 클랙스
from nltk.probability import MLEProbDist #최대 우도 추정값을 도출클래스
#.generate() 샘플 추출 (임의의값을 추출)

1.데이터수집

In [43]:
from nltk.corpus import movie_reviews
data=movie_reviews.sents()#단어별 토큰화된 문장들

2.데이터  전처리#모델 입력에 맞게 가공하는것<br> 
    - 토큰화<br>
    - 정형화<br>
    - 정규화<br>

In [44]:
data_l=[]#처리된 문서
n=0
for i in data:#i는 문장
    bg=ngrams(i,2,pad_left=True,pad_right=True,left_pad_symbol="SS",right_pad_symbol="SE")    
    data_l+=[t for t in bg]

3.모델 학습

In [45]:
cfd= ConditionalFreqDist(data_l)
cpd = ConditionalProbDist(cfd,MLEProbDist)

4.검증(생략)

5.동작

최종동작
-(data별 추가적 처리가 필요)

In [48]:
st='SS'
all_str=[]
import random
random.seed(10)
while True:    
    st=cpd[st].generate()
    all_str.append(st)
    if st == 'SE':
        all_str.pop()
        break
생성된_data=''.join(all_str)

In [49]:
생성된_data

'sheandfineeffect;frankly,anddoinghissonbecomesshockinglylazyshortcuttohermotherof"story"'